# Simulating quantum programs on GPUs

In this notebook, you will learn how to simulate quantum circuits using GPUs with NVIDIA CUDA-Q and Braket Hybrid Jobs.

We start with necessary imports that are used in the examples below.

In [ ]:
import time

from random_circuits import random_circuit_generator_factory

from braket.aws import AwsSession
from braket.jobs import hybrid_job
from braket.jobs.config import InstanceConfig
from braket.jobs.environment_variables import get_job_device_arn
from braket.jobs.image_uris import Framework, retrieve_image

For this example, we will use the CUDA-Q hybrid jobs container provided by Braket.

In [ ]:
image_uri = retrieve_image(Framework.CUDAQ, AwsSession().region)

## Running hybrid jobs on GPUs
To use GPUs for circuit simulation, you can set the target backend to `nvidia`. You also need to select an instance that has NVIDIA GPUs. In the code snippet below, the instance type "ml.p3.2xlarge" is used as an example. The instance type "ml.p3.2xlarge" has a single NVIDIA V100 GPU. You can check [this page](https://aws.amazon.com/braket/pricing/) and [this page](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-configure-job-instance-for-script.html) to view available instances for Braket Hybrid Jobs. The `include_modules` keyword is set to `random_circuits` in order to use the random circuit generator defined in "random_circuits.py".

In [ ]:
@hybrid_job(
    device="local:nvidia/nvidia",
    image_uri=image_uri,
    include_modules="random_circuits",
    instance_config=InstanceConfig(instanceType="ml.p3.2xlarge"),
)
def gpu_job(n_qubits, n_gates, n_terms, n_shots):
    import cudaq

    # Define backend
    device = get_job_device_arn()
    cudaq.set_target(device.split("/")[-1])
    print("CUDA-Q backend: ", cudaq.get_target())

    # Define circuit and observables
    get_random_circuit = random_circuit_generator_factory()
    circuit = get_random_circuit(n_qubits, n_gates)
    hamiltonian = cudaq.SpinOperator.random(n_qubits, n_terms)

    # Time the circuit simulation
    t0 = time.time()
    result = cudaq.observe(circuit, hamiltonian, shots_count=n_shots)
    t1 = time.time()
    print(f"result: {result.expectation()} | duration: {t1 - t0}")

When the `gpu_job` function is called, it creates a Braket Hybrid Job that runs on AWS, performing circuit simulations with the GPU.

In [ ]:
n_qubits = 20
n_gates = 150
n_terms = 100
n_shots = 1000

job = gpu_job(n_qubits, n_gates, n_terms, n_shots)
print("Job ARN: ", job.arn)

## Tensor network backend
The `nvidia` backend shown in the example above is a state vector simulator. CUDA-Q also supports tensor network simulation with the `tensornet` backend. You can view [this page](https://nvidia.github.io/cuda-quantum/latest/using/backends/simulators.html#tensor-network-simulators) to learn more about the tensor network simulator. The list of CUDA-Q backends can be viewed in the [CUDA-Q documentation](https://nvidia.github.io/cuda-quantum/latest/using/backends/backends.html).

In [ ]:
@hybrid_job(
    device="local:nvidia/tensornet",
    image_uri=image_uri,
    include_modules="random_circuits",
    instance_config=InstanceConfig(instanceType="ml.p3.2xlarge"),
)
def gpu_tn_job(n_qubits, n_gates, n_terms, n_shots):
    import cudaq

    # Define backend
    device = get_job_device_arn()
    cudaq.set_target(device.split("/")[-1])
    print("CUDA-Q backend: ", cudaq.get_target())

    # Define circuit and observables
    get_random_circuit = random_circuit_generator_factory()
    circuit = get_random_circuit(n_qubits, n_gates)
    hamiltonian = cudaq.SpinOperator.random(n_qubits, n_terms)

    # Time the circuit simulation
    t0 = time.time()
    result = cudaq.observe(circuit, hamiltonian, shots_count=n_shots)
    t1 = time.time()
    print(f"result: {result.expectation()} | duration: {t1 - t0}")

When circuits are shallow, the tensor network simulator can run circuits with high qubit count. For example, the code snippet below runs a simulation of 50 qubits, and the simulation finishes in a few seconds.

In [ ]:
n_qubits = 40
n_gates = 100
n_terms = 10
n_shots = 1000

tn_job = gpu_tn_job(n_qubits, n_gates, n_terms, n_shots)
print("Tensornet Job ARN: ", tn_job.arn)

You can download results from both hybrid jobs with:

In [ ]:
tn_result = tn_job.result()

## Summary
This notebook shows you how to target CUDA-Q GPU simulators for circuit simulation. In particular, this notebook shows an example of using a CUDA-Q state vector simulator, the `nvidia` backend. If you have shallow circuits with high qubit count, you can use the CUDA-Q tensor network simulator, the `tensornet` backend, which may execute your circuits faster.